Import และ config path

In [2]:
pip install joblib

Defaulting to user installation because normal site-packages is not writeable
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install tensorflow keras numpy matplotlib scikit-learn opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 1.6/331.9 MB 9.3 MB/s eta 0:00:36
   ---------------------------------------- 3.1/331.9 MB 8.0 MB/s eta 0:00:42
    --------------------------------------- 4.7/331.9 MB 7.9 MB/s eta 0:00:42
    --------------------------------------- 5.5/331.9 MB 7.5 MB/s eta 0:00:44
    --------------------------------------- 7.1/331.9 MB 7.3 MB/s eta 0:00:45
   - -------------------------------------- 8.9/331.9 MB 7.5 MB/s eta 0:00:44
   - -------------------------------------- 10.5/331.9 MB 7.4 MB/s eta 0:00:44
   - -------------------------------------- 11.8/331.9 MB 7.4 MB/s eta 0:00:44
   - -------------------------------------- 13.6/331.9 MB 7.5 MB/s eta 0:00:43
   - --------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os, cv2, numpy as np, joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import Adam

CHARS74K_ROOT = r"English/Img/GoodImg/Bmp"  # มี Sample001, Sample002, ... 62 folders [web:50]

IMG_SIZE = 32

โหลดรูปจาก Chars74K

In [8]:
def load_chars74k(root_dir, limit_per_class=None):
    X, y = [], []
    class_dirs = sorted(os.listdir(root_dir))  # Sample001, Sample002, ...
    
    for cdir in class_dirs:
        cpath = os.path.join(root_dir, cdir)
        if not os.path.isdir(cpath):
            continue
        
        files = [f for f in os.listdir(cpath) if f.lower().endswith('.png')]
        if limit_per_class:
            files = files[:limit_per_class]
        
        # label เป็นตัวอักษรจากชื่อไฟล์ เช่น 'img001-00001.png' → ใช้ตัวแรกในชื่อ class mapping ภายหลัง
        for fname in files:
            fpath = os.path.join(cpath, fname)
            img = cv2.imread(fpath)
            if img is None:
                continue
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X.append(img)
            # label เป็นชื่อโฟลเดอร์ (Sample0xx) ไว้ไป map ต่อทีหลัง
            y.append(cdir)
    return np.array(X), np.array(y)

X, y_folder = load_chars74k(CHARS74K_ROOT, limit_per_class=300)  # ให้เร็วขึ้น
print(X.shape, y_folder.shape)

(7011, 32, 32, 3) (7011,)


map โฟลเดอร์เป็นตัวอักษร

In [ ]:

FOLDER_TO_CHAR = {
    "Sample001": "0",
    "Sample002": "1",
    "Sample003": "2",
    "Sample004": "3",
    "Sample005": "4",
    "Sample006": "5",
    "Sample007": "6",
    "Sample008": "7",
    "Sample009": "8",
    "Sample010": "9",
    "Sample011": "A",
    "Sample012": "B",
    "Sample013": "C",
    "Sample014": "D",
    "Sample015": "E",
    "Sample016": "F",
    "Sample017": "G",
    "Sample018": "H",
    "Sample019": "I",
    "Sample020": "J",
    "Sample021": "K",
    "Sample022": "L",
    "Sample023": "M",
    "Sample024": "N",
    "Sample025": "O",
    "Sample026": "P",
    "Sample027": "Q",
    "Sample028": "R",
    "Sample029": "S",
    "Sample030": "T",
    "Sample031": "U",
    "Sample032": "V",
    "Sample033": "W",
    "Sample034": "X",
    "Sample035": "Y",
    "Sample036": "Z",
    "Sample037": "a",
    "Sample038": "b",
    "Sample039": "c",
    "Sample040": "d",
    "Sample041": "e",
    "Sample042": "f",
    "Sample043": "g",
    "Sample044": "h",
    "Sample045": "i",
    "Sample046": "j",
    "Sample047": "k",
    "Sample048": "l",
    "Sample049": "m",
    "Sample050": "n",
    "Sample051": "o",
    "Sample052": "p",
    "Sample053": "q",
    "Sample054": "r",
    "Sample055": "s",
    "Sample056": "t",
    "Sample057": "u",
    "Sample058": "v",
    "Sample059": "w",
    "Sample060": "x",
    "Sample061": "y",
    "Sample062": "z"
    
}

y_chars = np.array([FOLDER_TO_CHAR[f] for f in y_folder])

# encode เป็นตัวเลข 0..C-1
le = LabelEncoder()
y_idx = le.fit_transform(y_chars)
num_classes = len(le.classes_)
print("Classes:", le.classes_, "Total:", num_classes)


X = X.astype("float32") / 255.0
y_onehot = to_categorical(y_idx, num_classes=num_classes)

X_train, X_val, y_train, y_val = train_test_split(X, y_onehot, test_size=0.2, random_state=42, stratify=y_idx)
print(X_train.shape, X_val.shape)


Classes: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z'
 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r'
 's' 't' 'u' 'v' 'w' 'x' 'y' 'z'] Total: 62
(5608, 32, 32, 3) (1403, 32, 32, 3)


 สร้างและฝึก CNN

In [ ]:

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=64, epochs=300, validation_data=(X_val, y_val))


Epoch 1/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.0380 - loss: 3.9582 - val_accuracy: 0.0428 - val_loss: 3.8869
Epoch 2/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.0403 - loss: 3.8958 - val_accuracy: 0.0442 - val_loss: 3.8739
Epoch 3/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.0394 - loss: 3.8907 - val_accuracy: 0.0428 - val_loss: 3.8565
Epoch 4/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.0451 - loss: 3.8760 - val_accuracy: 0.0492 - val_loss: 3.8273
Epoch 5/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.1307 - loss: 3.5895 - val_accuracy: 0.3414 - val_loss: 2.8486
Epoch 6/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3445 - loss: 2.5545 - val_accuracy: 0.5403 - val_loss: 1.9467
Epoch 7/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.4982 - loss: 1.8195 - val_accuracy: 0.6614 - val_loss: 1.2015
Epoch 8/300
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.5949 - loss: 1.4050 - val_accuracy: 0.

 เซฟโมเดลและ label mapping

In [ ]:
import os, joblib

os.makedirs("models", exist_ok=True)
model.save("models/chars74k_model.h5")
# เซฟ list ของ class เช่น ['0','1','2',...,'A',...]
joblib.dump(le.classes_.tolist(), "models/chars74k_labels.pkl")
print("Saved model and labels to models/")
# บันทึกโมเดลที่ฝึกแล้วและ mapping ของ labels
# - สร้างโฟลเดอร์ models/ ถ้ายังไม่มี
# - เซฟไฟล์โมเดล .h5 และไฟล์ labels (list ของคลาสตามลำดับ output ของ softmax)
import os, joblib

os.makedirs("models", exist_ok=True)
model.save("models/chars74k_model.h5")
# เซฟ list ของ class เช่น ['0','1','2',...,'A',...] (ใช้เมื่อนำโมเดลไปใช้งาน)
joblib.dump(le.classes_.tolist(), "models/chars74k_labels.pkl")
print("Saved model and labels to models/")


Saved model and labels to models/
